In [1]:
from coremltools.models import datatypes
from coremltools.models.neural_network import NeuralNetworkBuilder
import coremltools as ct
import torch

Torch version 2.2.2 has not been tested with coremltools. You may run into unexpected errors. Torch 2.1.0 is the most recent version that has been tested.


In [2]:
tf_model = torch.load('Files/YOLO_model_traced.pt')


/Users/egorlogutov/Desktop/Github/CoreML_converter/venv/lib/python3.11/site-packages/torch/serialization.py:1007: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn("'torch.load' received a zip file that looks like a TorchScript archive"


In [9]:
import coremltools as ct
from coremltools.models.neural_network import quantization_utils
from coremltools.models import datatypes
from coremltools.models import neural_network as nn

# Создание объекта для описания входного изображения
image_input = ct.ImageType(name="input_image", shape=(1, 3, 640, 640), 
                           bias=[-1,-1,-1], scale=1/127.5)


mlmodel = ct.convert(
    tf_model,
    inputs=[image_input]
)

# Сохранение модели в формате Core ML
mlmodel.save('yolov5.mlpackage')


When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_targer' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details please see the link: https://coremltools.readme.io/docs/unified-conversion-api#target-conversion-formats
Support for converting Torch Script Models is experimental. If possible you should use a traced model for conversion.
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 579.43 passes/s]


In [ ]:
import cv2
import numpy as np

def draw_boxes(image, pred):
    for *xyxy, conf, cls in reversed(pred):
        label = f'{model.names[int(cls)]} {conf:.2f}'
        xyxy = [int(x.item()) for x in xyxy]
        cv2.rectangle(image, xyxy[0:2], xyxy[2:4], (255, 0, 0), 2)
        cv2.putText(image, label, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    return image


In [10]:
model =  torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)


Using cache found in /Users/egorlogutov/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-4-22 Python-3.11.9 torch-2.2.2 CPU

100%|██████████| 14.1M/14.1M [00:10<00:00, 1.40MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [13]:
import torch
import coremltools as ct
import cv2
import numpy as np

class YOLOv5Wrapper(torch.nn.Module):
    def __init__(self, model):
        super(YOLOv5Wrapper, self).__init__()
        self.model = model

    def forward(self, x):
        preds = self.model(x)
        # Получаем изображение обратно в формате OpenCV для постобработки
        result_img = (x[0].permute(1, 2, 0).detach().cpu().numpy() * 255).astype(np.uint8)
        result_img = cv2.cvtColor(result_img, cv2.COLOR_RGB2BGR)
        return self.draw_boxes(result_img, preds.xyxy[0])

    def draw_boxes(self, image, pred):
        for *xyxy, conf, cls in reversed(pred):
            label = f'{self.model.names[int(cls)]} {conf:.2f}'
            xyxy = [int(x.item()) for x in xyxy]
            cv2.rectangle(image, xyxy[0:2], xyxy[2:4], (255, 0, 0), 2)
            cv2.putText(image, label, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        return image

# Загружаем модель YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.eval()

# Оборачиваем модель
wrapped_model = YOLOv5Wrapper(model)
traced_model = torch.jit.trace(wrapped_model, torch.rand(1, 3, 640, 640))

# Конвертация в Core ML
image_input = ct.ImageType(name="input_image", shape=(1, 3, 640, 640), scale=1/255.0, bias=[0, 0, 0])
mlmodel = ct.convert(traced_model, inputs=[image_input])
mlmodel.save('yolov5_image_output.mlmodel')


Using cache found in /Users/egorlogutov/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-4-22 Python-3.11.9 torch-2.2.2 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/Users/egorlogutov/.cache/torch/hub/ultralytics_yolov5_master/models/yolo.py:100: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.dynamic or self.grid[i].shape[2:4] != x[i].shape[2:4]:
/var/folders/2m/1nfzrjzx5rjdbx6qbmzt0bjw0000gp/T/ipykernel_76995/149577676.py:14: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs

AttributeError: 'Tensor' object has no attribute 'xyxy'

Большинство моделей хоть и берут ImageInput но возвращают не изображение а ограничевающие рамки которые прийдется накладывать самостоятельно

чтобы было автоматически и при конвертации тоже задавать на выход image type необходимо в модель это записать тоесть создать доп слой модели что то типа wrappera или автоэнкодера который будет оборачивать модель и на выходедаст не координаты рамок а стразу фото вместе с рамками